In [1]:
import os
import glob
import warnings
import pandas as pd

pd.options.display.float_format = "{:,.2f}".format
pd.set_option("mode.chained_assignment", None)

warnings.filterwarnings("ignore")

In [2]:
# Combine all the CSV files
path = r"../data"
all_csv = glob.glob(path + "/*.csv")

temp = []
for csv in all_csv:
    df = pd.read_csv(csv, index_col=0, header=0, thousands=",")
    temp.append(df)

df = pd.concat(temp, axis=0, ignore_index=True)
df.head()

,Service,Type,Rating,Hires,Price,Zip Code
0,"MSF Landscapes, Masonry, and Irrigation",47,4.50,49.00,NaN,2903
1,Emi Ferreira Masonry,47,5.00,72.00,NaN,2903
2,27 Newman St,47,5.00,NaN,NaN,2903
3,JFLANDSCAPING,47,5.00,NaN,NaN,2903
4,Leaves land scape inc.,47,3.50,4.00,NaN,2903


In [3]:
df.shape

(79304, 6)

In [4]:
# Check and fix dtype
df.dtypes

Service      object
Type          int64
Rating      float64
Hires       float64
Price        object
Zip Code      int64
dtype: object

In [5]:
# See some prices values
df["Price"].unique()[:19]

# Need to fix that hour thing

array([nan, '75/hour', '60/hour', '495/hour', '28/hour', '50/hour',
       '65/hour', '45/hour', '125/hour', '35/hour', '40/hour', '70/hour',
       '39/hour', '55/hour', '85/service', '50/service', '150-$200', 75.0,
       300.0], dtype=object)

In [6]:
# See different types of payment
df["Price"] = df["Price"].astype(str)

tempList = []
checkUnique = []
for i in df["Price"]:
    x = i.split("/")
    tempList.append(x)

for i in tempList:
    try:
        x = i[1]
        checkUnique.append(x)
    except:
        pass

checkUnique = list(dict.fromkeys(checkUnique))
checkUnique

['hour',
 'service',
 'session',
 'on-site',
 'logo',
 'lesson',
 'consult',
 'walk',
 'cat',
 'person',
 'visit']

In [7]:
# Removed the hour into another column
tempPrice = []
tempPriceType = []
for i in df["Price"]:
    if i.find("-") < 0:
        x = i.split("/")
        tempPrice.append(x[0])
        try:
            y = x[1]
        except:
            y = ""
    else:
        x = i.split("/")
        x = x[0].split("-")
        tempPrice.append(x[0])
        try:
            y = x[1]
        except:
            y = ""
    tempPriceType.append(y)

# Add it to dataframe and check
df["Price"] = tempPrice
df["Price Type"] = tempPriceType
df["Price Type"] = df["Price Type"].astype(str)

# Convert price column to float
df["Price"] = df["Price"].str.replace(",", "").astype(float)

In [8]:
# Double check data types
df.dtypes

Service        object
Type            int64
Rating        float64
Hires         float64
Price         float64
Zip Code        int64
Price Type     object
dtype: object

In [9]:
# Add the service type from database
serviceDf = pd.read_csv("../src/all_services.csv")

dfMerge = df.rename(columns={"Type": "ID"})
dfMerge = pd.merge(dfMerge, serviceDf, left_on="ID", right_on="ID", how="left").drop(
    "ID", axis=1
)
df = dfMerge

In [10]:
zipCodes = pd.read_csv("../src/us_states.csv")

dfMerge = df
dfMerge = pd.merge(
    dfMerge,
    zipCodes,
    left_on="Zip Code",
    right_on="Representative ZIP Code",
    how="left",
).drop("Representative ZIP Code", axis=1)

df = dfMerge

In [11]:
df.sample(3)

,Service,Rating,Hires,Price,Zip Code,Price Type,Services,Type,Segments,Capital City,State,Abbreviation
52601,CR INSPIRES,5.00,NaN,NaN,36104,,Life Coaching,Main,Wellness,Montgomery,Alabama,AL
74104,Greyhawk Construction,5.00,3.00,NaN,95112,,Basement Finishing or Remodeling,Peripheral,Home Improvement,San Jose,California,CA
75460,Ask The Organizing Diva,5.00,8.00,65.00,2201,hour,Closet and Shelving System Installation,Peripheral,Home Improvement,Boston,Massachusetts,MA


In [12]:
# See shape
df.shape

(79304, 12)

In [13]:
# Check for duplicates
dfDuplicate = df[~df.duplicated("Service", keep="first")]
df = dfDuplicate

In [14]:
df.shape

(18169, 12)

In [15]:
# See overview of data
df.describe()

,Rating,Hires,Price,Zip Code
count,"17,004.00","12,962.00","10,429.00","18,169.00"
mean,4.89,47.31,127.21,"52,037.80"
std,0.31,99.29,251.46,"31,855.06"
min,1.00,2.00,1.00,"2,201.00"
25%,5.00,6.00,50.00,"23,219.00"
50%,5.00,17.00,80.00,"50,309.00"
75%,5.00,48.00,135.00,"83,702.00"
max,5.00,"2,380.00","7,850.00","99,801.00"


In [16]:
# Get implied revenue
df["Implied Revenue"] = df["Price"] * df["Hires"]

In [17]:
# Fix Zip Code datatype
df["Zip Code"] = df["Zip Code"].astype(str)

In [18]:
# See columns
df.columns

Index(['Service', 'Rating', 'Hires', 'Price', 'Zip Code', 'Price Type',
       'Services', 'Type', 'Segments', 'Capital City', 'State', 'Abbreviation',
       'Implied Revenue'],
      dtype='object')

In [19]:
# Rearrange columsn
df = df[
    [
        "Segments",
        "Services",
        "Type",
        "Service",
        "Hires",
        "Price",
        "Price Type",
        "Rating",
        "Zip Code",
        "Capital City",
        "State",
        "Abbreviation",
        "Implied Revenue",
    ]
]

In [20]:
df.to_csv("../output/all_services.csv")